In [29]:
import pandas as pd
import os
location = str(os.getcwd())

In [30]:
f = (location[:-29] + "Tweets\\tweets_cleaned.csv")
df = pd.read_csv(f)
df = df.to_dict("index")
print(len(df))

45982


In [31]:
#cleaning body text
data = []

for i in range(len(df)):
    text_clean = ""
    text = df[i]["text"]
    text = text.split(" ")
    for item in text:
        if "@" not in item:
            text_clean += item
            text_clean += " "
    if text_clean not in data:
        data.append(text_clean)
print(len(data))

21027


In [11]:
import re
import pandas as pd
import nltk

Tokenizing

In [14]:
def list_tokens(Data):
    """
    This function will convert Articles into list of tokens(either alpha or numeric)
    
    arguments:
        input_text: "Data" of type "pandas Dataframe". 
                    
    return:
        value: "list_of_tokens" after splitting all the articles in 
                "Processed_Content" attribute into list of tokens.
        
    Example:
    Input : This is a nice place to live. 
    Output : ['This', 'is', 'a', 'nice', 'place', 'to', 'live']
    """
    list_of_tweets = []
    # For each article in Processed_Content
    for tweet in Data:
        # split each article into tokens of words either alpha or numeric (not a single punctuation)
        tokens = re.findall("[\w']+", tweet) # [\w]+ --> Checks for more than one occurrence of words which are either alpha or numeric.
        # Appending each list of tokens for each article.
        list_of_tweets.append(tokens)  
    # Unlisting list of list elements to only one list having all the tokens for each & every article.
    list_of_tokens = [item for items in list_of_tweets for item in items]
    return list_of_tokens
list_of_tokens = list_tokens(data)
print(len(list_of_tokens))

455583


Running Noise Removal

In [15]:
en_stopwords = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

In [19]:
tokens = []
for item in list_of_tokens:
    tokens.append(item.lower())

In [20]:
tokens_filtered = []
for item in tokens:
    if item not in en_stopwords:
        tokens_filtered.append(item)

print(len(tokens_filtered))

322670


removing tokens that do not register as words

In [ ]:
print(len(tokens_filtered))

In [21]:
pd.DataFrame(tokens_filtered).to_csv('tokens_filtered.csv')

In [22]:
#get list from csv so I don't have to refilter
f = (location + "\\tokens_filtered.csv")
df = pd.read_csv(f)

In [26]:
tokens_filtered = df['0'].tolist()
tokens_filtered = set(tokens_filtered)
tokens_filtered = list(tokens_filtered)
print(len(tokens_filtered))

33558


In [32]:
#finding how often each token appears
from tqdm import tqdm
tweets = data
data = None

token_frequency = {}

for word in tokens_filtered:
    token_frequency[word] = 0

for i in tqdm(range(len(tweets))):
    tweet = tweets[i]
    for word in tokens_filtered:
        current = int(token_frequency[word])
        new = current + tweet.count(str(word))
        token_frequency[word] = new

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21027/21027 [14:58<00:00, 23.39it/s]


In [33]:
token_freq_list = []
for word in tokens_filtered:
    token_freq_list.append({"token": word, "frequency": token_frequency[word]})
    
pd.DataFrame(token_freq_list).sort_values(by=['frequency'], ascending=False)

,token,frequency
14599,e,197808
9936,o,152807
16826,n,134944
28793,r,113166
24480,l,83102
...,...,...
13510,uiepeaor9w,0
13509,yhyqrrowqf,0
13508,dmsqxxemw8,0
13507,kwyzmrhgmk,0


In [34]:
from english_words import english_words_set

df = []
for item in token_freq_list:
    if len(str(item["token"])) >= 5:
        #check if token is word
        if item["token"] in english_words_set:
            df.append(item)
token_freq_list = df
print(len(token_freq_list))
token_freq_list = pd.DataFrame(token_freq_list).sort_values(by=['frequency'], ascending=False)

2947


In [35]:
token_freq_list.head(30)

,token,frequency
1177,sexual,10429
546,assault,8204
183,student,3723
1713,harass,1759
2562,convict,1475
2670,campus,1305
1062,school,1244
2348,month,1244
1853,conduct,1240
1806,story,1083


In [36]:
pd.DataFrame(token_freq_list).to_csv('tokens_filtered_freq.csv')

In [37]:
#Import from csv to list of dicts
f = (location + "\\tokens_filtered_freq.csv")
df = pd.read_csv(f)

token_freq_list = []
for item in (df.values.tolist()[1:]):
    token_freq_list.append({"token": item[1], "frequency": item[2]})
print(len(token_freq_list))

2946
